In [49]:
import pandas as pd 
import numpy  as np  
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram    
import sys 
sys.setrecursionlimit(10000)  #increasing maximum recursion from 999 to 10K

from scipy.cluster.hierarchy import dendrogram, fcluster, leaves_list
from scipy.spatial import distance
from fastcluster import linkage
from sklearn.metrics.pairwise import pairwise_distances  
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage , fcluster
from scipy.cluster.hierarchy import cophenet
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation 

import scipy.stats
import math 

# Reading diabetes2 File

In [50]:
fields=['new_euid','claim','dx1','dx2','dx3']
type= {'new_euid':str,'dx1':str,'dx2':str,'dx3':str,'claim':str}
diabetic_patients = pd.read_csv("../../Data/Diabetic_Patient_Clustering/diabetes2.csv",usecols=fields,dtype=type)
diabetic_patients.head() 

,new_euid,claim,dx1,dx2,dx3
0,33,1,99520,4580,5939
1,33,7,20410,NaN,NaN
2,33,1,20280,NaN,NaN
3,33,1,9895,78605,NaN
4,33,1,25000,3671,3674


In [51]:
len(diabetic_patients['new_euid'].unique()) #Unique Patients

8955

In [97]:
print("Total Observations")
print(diabetic_patients.shape)                    # Total observatons

print("\n")  

print("Null observation per column")
print(diabetic_patients.isnull().sum())           # Null observation per column  

print("\n") 

print("unique  patients")
print(len(diabetic_patients['new_euid'].unique()))  #Total patients

Total Observations
(62194, 5)


Null observation per column
new_euid        0
claim           0
dx1             0
dx2         32621
dx3         44229
dtype: int64


unique  patients
8955


# Importing Diagnosis Code

In [53]:
diag_code=pd.read_csv('../../Data/Diabetic_Patient_Clustering/dx_ccs_mapping.csv')

ccs_codes=pd.read_csv('../../Data/ccs_code.csv')  #reading ccs code
ccs_codes['ccs_code']= "CCS"+ ccs_codes.START.map(str) #Adding CCS Codes 


In [54]:
diag_code.dtypes

ICD-9-CM-CODE               object
CCS-CATEGORY                 int64
CCS-CATEGORY-DESCRIPTION    object
dtype: object

In [55]:
len(diag_code['CCS-CATEGORY'].unique())  ##All CCS Codes

283

In [56]:
diag_code['ccs_code']= "CCS"+ diag_code['CCS-CATEGORY'].map(str) #Adding CCS Codes 
diag_code.head() 

,ICD-9-CM-CODE,CCS-CATEGORY,CCS-CATEGORY-DESCRIPTION,ccs_code
0,01000,1,Tuberculosis,CCS1
1,01001,1,Tuberculosis,CCS1
2,01002,1,Tuberculosis,CCS1
3,01003,1,Tuberculosis,CCS1
4,01004,1,Tuberculosis,CCS1


In [57]:
#cleaning ICD-9-CM-CODE
diag_code['ICD-9-CM-CODE']=diag_code['ICD-9-CM-CODE'].map(lambda x:x.strip()) 

# Merging Diabetic_Patients and diag_code to get new_euid CCS-mapping

In [58]:
x=pd.merge(diag_code,diabetic_patients,left_on='ICD-9-CM-CODE',right_on='dx1',how='right')
x.drop(['ICD-9-CM-CODE','ccs_code','CCS-CATEGORY-DESCRIPTION','dx1'],axis=1,inplace=True)
x=x.rename(columns={'CCS-CATEGORY':'ccs_dx1'})  



x=pd.merge(diag_code,x,left_on='ICD-9-CM-CODE',right_on='dx2',how='right')
x.drop(['ICD-9-CM-CODE','CCS-CATEGORY-DESCRIPTION','ccs_code','dx2'],axis=1,inplace=True)
x=x.rename(columns={'CCS-CATEGORY':'ccs_dx2'})  

x=pd.merge(diag_code,x,left_on='ICD-9-CM-CODE',right_on='dx3',how='right')
x.drop(['ICD-9-CM-CODE','CCS-CATEGORY-DESCRIPTION','ccs_code','dx3'],axis=1,inplace=True)
x=x.rename(columns={'CCS-CATEGORY':'ccs_dx3'})  

diabetic_patients_ccs=x

In [59]:
diabetic_patients_ccs.head()

,ccs_dx3,ccs_dx2,ccs_dx1,new_euid,claim
0,2.0,18.0,151,4825,1
1,2.0,159.0,50,464784,3
2,2.0,159.0,163,7754,1
3,2.0,199.0,50,706898,1
4,2.0,201.0,100,661205,1


In [60]:
del diabetic_patients_ccs['claim']
print(diabetic_patients_ccs.shape)
print(diabetic_patients_ccs.isnull().sum())

(62194, 4)
ccs_dx3     44233
ccs_dx2     32626
ccs_dx1         0
new_euid        0
dtype: int64


In [61]:
#Rearranging columns
cols=diabetic_patients_ccs.columns.tolist()
cols=cols[-1:] + cols[:-1]
diabetic_patients_ccs=diabetic_patients_ccs[cols]

In [62]:
diabetic_patients_ccs.head() 

,new_euid,ccs_dx3,ccs_dx2,ccs_dx1
0,4825,2.0,18.0,151
1,464784,2.0,159.0,50
2,7754,2.0,159.0,163
3,706898,2.0,199.0,50
4,661205,2.0,201.0,100


In [63]:
diabetic_patients_ccs=diabetic_patients_ccs.fillna(0)

# Making Binary Matrix

In [64]:
cols=list(ccs_codes['ccs_code'])
cols.insert(0,'new_euid') 
diabetic_patients_binary=pd.DataFrame(columns=cols,index=diabetic_patients.index) 
diabetic_patients_binary.head()  

,new_euid,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
##Writing Code to fill up binary Matrix  
for index,row in diabetic_patients_ccs.iterrows(): 
    if(row['ccs_dx1'] !=0): 
        col="CCS" +str(row['ccs_dx1'])
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid']
    if(row['ccs_dx2'] !=0): 
        col="CCS" +str(row['ccs_dx2'])
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid']
    if(row['ccs_dx3'] !=0): 
        col="CCS" +str(row['ccs_dx3']) 
        diabetic_patients_binary.loc[index][col]=1 
        diabetic_patients_binary.loc[index]['new_euid']=row['new_euid'] 
    

In [66]:
print(diabetic_patients_binary.shape)  
print(diabetic_patients_ccs.shape)

(62194, 284)
(62194, 4)


In [67]:
diabetic_patients_binary=diabetic_patients_binary.fillna(0)
diabetic_patients_binary=diabetic_patients_binary.groupby(['new_euid']).sum()
print(diabetic_patients_binary.shape)  

(8955, 283)


In [68]:
diabetic_patients_binary.head() 

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
100114,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
10013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10014,0,0,2,2,6,0,0,0,3,0,...,0,0,0,0,0,3,0,1,0,0
100397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
## changing all values greater >1 =1
def f1(x):
    if x>=1:
        return 1
    else:
        return 0

diabetic_patients_binary=diabetic_patients_binary.applymap(f1)
diabetic_patients_binary.head() 

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
100114,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
10013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10014,0,0,1,1,1,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
100397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
100438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
print(diabetic_patients_binary.shape)
del diabetic_patients_binary['CCS49'] ## Removing CCS49 i,e Diabetes with no complication
del diabetic_patients_binary['CCS256'] #Removing Exam/Eval attribute.
diabetic_patients_binary=diabetic_patients_binary.loc[:,(diabetic_patients_binary!=0).any(axis=0)] ## Removing 0 column  
print(diabetic_patients_binary.shape) 

(8955, 283)
(8955, 247)


# Making CCS_Stats_Summary

In [71]:
### Making CCS Codes with description and population statistics
df=diabetic_patients_binary
codes_stats=pd.DataFrame(df.sum())
codes_stats.reset_index(inplace=True) 
codes_stats=codes_stats.rename(columns={0:'Total_Patients'})
codes_stats.head()

,index,Total_Patients
0,CCS1,2
1,CCS10,1371
2,CCS100,67
3,CCS101,802
4,CCS102,627


In [72]:
ccs_codes.head()

,START,LABEL,ccs_code
0,1,Tuberculosis,CCS1
1,10,Immuniz/scrn,CCS10
2,100,Acute MI,CCS100
3,101,Coron athero,CCS101
4,102,Chest pain,CCS102


In [73]:
tmp1=ccs_codes[['ccs_code','LABEL']]
codes_stats=pd.merge(tmp1,codes_stats,left_on='ccs_code',right_on='index')
del codes_stats['index'] 
codes_stats=codes_stats.sort_values(['Total_Patients'],ascending=False)
codes_stats['population_percentage']=(codes_stats['Total_Patients']*1.0/8955)*100.0
codes_stats.head()  

,ccs_code,LABEL,Total_Patients,population_percentage
189,CCS50,DiabMel w/cm,2420,27.024009
1,CCS10,Immuniz/scrn,1371,15.309883
186,CCS47,Ot bnign neo,1301,14.528197
116,CCS211,Ot conn tiss,1160,12.953657
245,CCS98,HTN,1121,12.518146


# Deduplicating Binary Matrix (unique vectors)

In [74]:
df=diabetic_patients_binary  
df_new=pd.DataFrame(df.groupby(df.columns.tolist(),as_index=False).size())  
print(df_new.shape)
df_new.reset_index(inplace=True)  
df_new=df_new.rename(columns={0:'Total_Patients'})
df_new.sort_values(['Total_Patients'],ascending=False,inplace=True)
df_new.head()


(6676, 1)


,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,451
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,185
5611,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,124
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,70
366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,44


# Performing Clustering Using Cosine Similarity

In [77]:
# changing datatype of binary_IDF_ddup
diabetic_patients_binary_ddup=df_new
cols=diabetic_patients_binary_ddup.columns.tolist()
diabetic_patients_binary_ddup[cols]=diabetic_patients_binary_ddup[cols].astype(float)


##converting 0,1 to IDF's
for index,row in diabetic_patients_binary_ddup.iterrows():
    #print(index)
    for col in cols[:-1]:
         if(row[col]==1):
            fre=codes_stats[codes_stats['ccs_code']==col]['population_percentage']
            fre=(fre*1.0/100).iloc[0]
            #print("frequency is",fre)
            diabetic_patients_binary_ddup.loc[index][col]=np.log(1.0/fre)*1.0 
            #print("code is",col,"frequency is" ,fre,"IDF is",np.log(1.0/fre)*1.0,"Valu
diabetic_patients_binary_ddup.head()

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,451.0
141,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,185.0
5611,0.0,1.876672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,124.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.077991,0.0,70.0
366,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,44.0


In [79]:
print(codes_stats[codes_stats['ccs_code']=='CCS10'])  
print(math.log(1.0/0.153098))

print(codes_stats[codes_stats['ccs_code']=='CCS98'])  
print(math.log(1.0/0.12518))

  ccs_code         LABEL  Total_Patients  population_percentage
1    CCS10  Immuniz/scrn            1371              15.309883
1.87667703976
    ccs_code LABEL  Total_Patients  population_percentage
245    CCS98   HTN            1121              12.518146
2.07800257749


In [106]:
checkx=diabetic_patients_binary_ddup[1:]
X=pdist(checkx.ix[:,0:len(diabetic_patients_binary_ddup.columns)-1],metric='cosine')

Z = linkage(X,method='average') 
c,cd=cophenet(Z,X)
c

0.38789511137369298

In [107]:
max_d=0.2
clusters=fcluster(Z,max_d,criterion='distance')
clusters

clusters=pd.DataFrame(clusters)
clusters=clusters.rename(columns = {0:'clusters_id'})  
checkx_new=pd.concat([checkx.reset_index(),clusters], axis=1)  
cols = checkx_new.columns.tolist() 
cols = cols[-2:] + cols[:-2] 
checkx_new=checkx_new[cols] 
checkx_new.set_index('index',inplace=True)  
community=checkx_new.groupby(['clusters_id']).Total_Patients.sum().sort_values(ascending=False)
community[:20] 

clusters_id
4197    185.0
4172    154.0
4188     83.0
4184     61.0
4268     58.0
4158     54.0
4244     54.0
4192     41.0
4147     40.0
4718     40.0
4171     34.0
3943     33.0
4116     30.0
4154     30.0
1864     26.0
4222     26.0
4174     24.0
2895     23.0
4077     23.0
1360     22.0
Name: Total_Patients, dtype: float64

In [82]:
def z_score(ccs_code):
    
    p_1=cluster_stats[cluster_stats['ccs_code']==ccs_code]['Total_Patients']*1.0/community[cluster_stats['cluster_id'][0]]
    p_1=p_1.iloc[0]
    p_2=codes_stats[codes_stats['ccs_code']==ccs_code]['Total_Patients']*1.0/14845.0
    p_2=p_2.iloc[0]
    
    n_1 = community[cluster_stats['cluster_id'][0]]*1.0 #cluster Population
    n_2 = 14845.0  #Total Population 
  
    p =(n_1*p_1 + n_2*p_2)*1.0/(n_1 + n_2)
   
    num = (p_1-p_2)*1.0
    deno= math.sqrt(p*(1-p)*( (1.0/n_1) + (1.0/n_2) ))
    
    z=round((num*1.0/deno),2)  
    
    p_values = round((scipy.stats.norm.sf(abs(z))*2),4) #twosided P Tailed test
    
    return(z,p_values)

def clusterstatistic(cluster_id):
    #print(cluster_id)
    cluster=checkx_new[checkx_new['clusters_id']==cluster_id]
   
    cluster=cluster.loc[:,(cluster!=0).any(axis=0)]
    #cluster.head()

    ccs_codes=cluster.columns.tolist()
    ccs_codes=ccs_codes[2:]
    cluster_stats=pd.DataFrame(columns=['cluster_id','CCS_Code','Total_Patients'])
    for code in ccs_codes:
        patients=cluster[cluster[code]!=0].Total_Patients.sum() 
        #print("code is",code,"patients is",patients)
        tmp=pd.DataFrame(index=range(1,2),columns=['cluster_id','CCS_Code','Total_Patients']) 
        tmp.ix[:,0]=cluster_id
        tmp.ix[:,1]=code
        tmp.ix[:,2]=patients 
        cluster_stats=cluster_stats.append(tmp) 

    #cluster_stats.head()

    #cluster_stats=cluster_stats[1:]
    cluster_stats.sort_values(by='Total_Patients',ascending=False,inplace=True)
    #cluster_stats.head()

    cluster_stats['%community']=(cluster_stats['Total_Patients']*1.0/cluster['Total_Patients'].sum())*100.0
    #cluster_stats['%diabetic_population']=(cluster_stats['Total_Patients']*1.0/14845.0)*100.0
    #cluster_stats.head()

    #tmp1.head()

    cluster_stats=pd.merge(tmp1,cluster_stats,left_on='ccs_code',right_on='CCS_Code')
    #cluster_stats.head()

    del cluster_stats['CCS_Code']
    cluster_stats.sort_values(by='Total_Patients',ascending=False,inplace=True)
    return cluster_stats

In [111]:
community.shape

(5535L,)

In [112]:
cluster_stats=clusterstatistic(4184)
cluster_stats['%diabetic_population']=np.nan
cluster_stats['z_score']=np.nan
cluster_stats['p_value']=np.nan
for index,row in cluster_stats.iterrows(): 
    z,p=z_score(row['ccs_code'])
    cluster_stats.loc[index,'%diabetic_population']=codes_stats[codes_stats['ccs_code']==row['ccs_code']] .population_percentage.iloc[0] 
    cluster_stats.loc[index,'z_score']=z
    cluster_stats.loc[index,'p_value']=p  
cluster_stats.sort_values(by='z_score',ascending=False,inplace=True)   
cluster_stats

,ccs_code,LABEL,cluster_id,Total_Patients,%community,%diabetic_population,z_score,p_value
0,CCS126,Ot up rsp in,4184.0,61,100,10.061418,29.80,0.0000
1,CCS50,DiabMel w/cm,4184.0,20,32.7869,27.024009,3.47,0.0005


# Performing Clustering Using Jaccard

In [91]:
df=diabetic_patients_binary  
df_new=pd.DataFrame(df.groupby(df.columns.tolist(),as_index=False).size())  
print(df_new.shape)
df_new.reset_index(inplace=True)  
df_new=df_new.rename(columns={0:'Total_Patients'})
df_new.sort_values(['Total_Patients'],ascending=False,inplace=True)
df_new.head()  

(6676, 1)


,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,451
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,185
5611,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,124
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,70
366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,44


In [92]:
checkx=df_new[1:]
X=pdist(checkx.ix[:,0:len(checkx.columns)-2],metric='jaccard')
Z=linkage(X,method='average') 
c,cd=cophenet(Z,X)
c

0.31640011251623379

In [93]:
max_d=0.2
clusters=fcluster(Z,max_d,criterion='distance')
clusters

clusters=pd.DataFrame(clusters)
clusters=clusters.rename(columns = {0:'clusters_id'})  
checkx_new=pd.concat([checkx.reset_index(),clusters], axis=1)  
cols = checkx_new.columns.tolist() 
cols = cols[-2:] + cols[:-2] 
checkx_new=checkx_new[cols] 
checkx_new.set_index('index',inplace=True)  
community=checkx_new.groupby(['clusters_id']).Total_Patients.sum().sort_values(ascending=False)
community[:20] 

clusters_id
6168    185
4765    124
1955     70
5495     44
3897     44
709      43
5650     41
2208     34
4673     30
2885     30
6012     26
1745     25
3452     22
1284     21
1539     21
5607     20
949      20
5211     18
4473     17
4820     16
Name: Total_Patients, dtype: int64

In [96]:
cluster_stats=clusterstatistic(6168)
cluster_stats['%diabetic_population']=np.nan
cluster_stats['z_score']=np.nan
cluster_stats['p_value']=np.nan
for index,row in cluster_stats.iterrows():
    z,p=z_score(row['ccs_code'])
    cluster_stats.loc[index,'%diabetic_population']=codes_stats[codes_stats['ccs_code']==row['ccs_code']] .population_percentage.iloc[0] 
    cluster_stats.loc[index,'z_score']=z
    cluster_stats.loc[index,'p_value']=p 
cluster_stats

,ccs_code,LABEL,cluster_id,Total_Patients,%community,%diabetic_population,z_score,p_value
0,CCS50,DiabMel w/cm,6168.0,185,100,27.024009,29.89,0.0
